#### Import Libs

In [1]:
import os
import re
import numpy as np
import simpy
import time
from numpy import random
from math import floor
from matplotlib import pyplot as plt
from scipy.stats import expon
from scipy.stats import uniform
from scipy.stats import randint

<hr>

## Ejercicio 1

####  File Utils

In [2]:
def read_file(folder, filename):
    filepath = os.path.join(folder, filename)
    file = open(filepath, 'r')
    page = file.read()
    return page

In [3]:
link_regex = '<a href=\"http://(.*?).html'

#### Page Rank

In [4]:
def open_pages(folder):
    filenames = os.listdir(folder)
    page_names = list(map(lambda filename: filename.split('.')[0], filenames))
    pages = []
    for filename in filenames:        
        page = read_file(folder, filename)
        pages.append(page)
    return (pages, page_names)

def markov_row(page, page_names):
    # n pages
    n = len(page_names)
    
    # search links in the page
    matchs = re.findall(link_regex, page)
    
    # build adjacency
    row = []
    if len(matchs) == 0:
        row = [1/n, 1/n, 1/n, 1/n, 1/n, 1/n]
    else:
        for page_name in page_names:
            n_matchs = 0
            for match in matchs:
                if page_name == match:
                    n_matchs += 1
            row.append(n_matchs / len(matchs))
    
    return row

def markov_matrix(pages, page_names):
    matrix = np.zeros(shape=(len(page_names), len(page_names)))
    for index, page in enumerate(pages):
        row = markov_row(page, page_names)
        matrix[index] = row
    return matrix

def distribucion_estacionaria(A):
    A = [1,0,0,0,0,0]
    for i in range(1000):
        A = np.dot(A,matrix)
    return A

#### Run

In [6]:
import string
folder = 'paginas'
filenames = os.listdir(folder)

indexes=dict()
for filename in filenames: 
    
    filepath = os.path.join(folder, filename)

    text = open(filepath, 'r')

    # Create an empty dictionary
    d = dict()
    # Loop through each line of the file
    for line in text:
        # Remove the leading spaces and newline character
        line = line.strip()

        # Convert the characters in line to
        # lowercase to avoid case mismatch
        line = line.lower()

        # Remove the punctuation marks from the line
        line = line.translate(line.maketrans("", "", string.punctuation))

        # Split the line into words
        words = line.split(" ")

        # Iterate over each word in line
        for word in words:
            # Check if the word is already in dictionary
            if word in d:
                # Increment count of word by 1
                d[word] = d[word] + 1
            else:
                # Add the word to dictionary with count 1
                d[word] = 1

                
    indexes[filename.split('.')[0]] = d

def calculate_scores(word):
    scores=dict()
    for key in list(indexes.keys()):
        if indexes[key].get(word) is None:
            # si no encuentra la palabra directamente le pone puntaje 0
            scores[key] = 0
        else:
            # ponderacion basica entre 
            scores[key]=0.5* indexes[key].get(word) + 0.5*page_rank[key]
    return sorted(scores.items(), key=lambda item: item[1],reverse=True)

scores=calculate_scores("film")
print(scores)

scores=calculate_scores("action")
print(scores)

scores=calculate_scores("actor")
print(scores)

[('martinscorcese', 3.071428571428571), ('angelinajolie', 1.6428571428571426), ('bradpitt', 1.6224489795918364), ('jonvoight', 0.5918367346938773), ('jenniferaniston', 0.5510204081632651), ('robertdeniro', 0.520408163265306)]
[('robertdeniro', 0), ('jenniferaniston', 0), ('martinscorcese', 0), ('jonvoight', 0), ('angelinajolie', 0), ('bradpitt', 0)]
[('jonvoight', 2.0918367346938775), ('robertdeniro', 1.5204081632653061), ('bradpitt', 1.1224489795918364), ('martinscorcese', 1.0714285714285712), ('angelinajolie', 0.6428571428571426), ('jenniferaniston', 0)]


In [5]:
folder = 'paginas'
pages, page_names = open_pages(folder)
matrix = markov_matrix(pages, page_names)

print("Markov Matrix\n")
print(matrix)
print("\n")

page_rank = distribucion_estacionaria(matrix)
print("Page Rank\n")
aux = dict()

for i in range(len(page_names)):
    aux[page_names[i]]=page_rank[i]
    print(page_names[i][:13],"\t",page_rank[i])
page_rank = aux

# Sort key
def value_getter(item):
     return item[1]

sorted(aux.items(), key=value_getter,reverse=True)

Markov Matrix

[[0.         0.         1.         0.         0.         0.        ]
 [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
 [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
 [0.         0.         0.         0.         0.66666667 0.33333333]
 [0.         0.         0.         0.5        0.         0.5       ]
 [0.         0.25       0.25       0.         0.5        0.        ]]


Page Rank

robertdeniro 	 0.040816326530612144
jenniferanist 	 0.10204081632653036
martinscorces 	 0.14285714285714252
jonvoight 	 0.18367346938775467
angelinajolie 	 0.28571428571428503
bradpitt 	 0.24489795918367288


[('angelinajolie', 0.28571428571428503),
 ('bradpitt', 0.24489795918367288),
 ('jonvoight', 0.18367346938775467),
 ('martinscorcese', 0.14285714285714252),
 ('jenniferaniston', 0.10204081632653036),
 ('robertdeniro', 0.040816326530612144)]

$\begin{bmatrix}    0 & 0 & 1 & 0 & 0 & 0\\    1/5 & 1/5 & 1/5 & 1/5 & 1/5 & 1/5 \\  1/5 & 1/5 & 1/5 & 1/5 & 1/5 & 1/5 \\   0 & 0 & 0 & 0 & 2/3 & 1/3 \\ 0 & 0 & 0 & 1/2 & 0 & 1/2 \\ 0 & 1/4 & 1/4 & 0 & 1/2 & 0 \\ \end{bmatrix}$

## Ejercicio 3

<b>Web Service</b>. El cual consulta una <b>Base de Datos</b>.

</b>Requests</b> siguen una distribucion exponencial de media 4.

$$
\mu = 4
$$

Dos opciones: <br>
a) Utilizar dos bases de datos => <b>M/M/2</b> (dos canales de atencion) <br>
b) Utilizar una base de datos => <b>M/M/1</b> (un canal de atencion) <br>

In [57]:
n = 100000

class Request:
    def __init__(self, timestamp):
        self.start_timestamp = timestamp
        
    def set_procesada(self, timestamp):
        self.end_timestamp = timestamp

class Requests:
    count_empty = 0
    
    def __init__(self, env, media, requests, requests_procesadas):
        self.env = env
        self.media = media
        self.requests = requests
        self.requests_procesadas = requests_procesadas

    def run(self):
        while len(self.requests_procesadas) < n:
            if len(self.requests) == 0:
                self.count_empty += 1
            request = Request(self.env.now)
            self.requests.append(request)
            tiempo = random.exponential(self.media)
            yield self.env.timeout(tiempo)
             
class BaseDeDatos():
    def __init__(self, env, media, requests, requests_procesadas):
        self.env = env
        self.media = media
        self.requests = requests
        self.requests_procesadas = requests_procesadas
    
    def run(self):
        while len(self.requests_procesadas) < n:
            if len(self.requests) > 0:
                request = self.requests.pop()
                request.set_procesada(self.env.now)
                self.requests_procesadas.append(request)
            tiempo = random.exponential(self.media)
            yield self.env.timeout(tiempo)

class WebService:

    # Requests
    requests = []
    requests_procesadas = []
    
    # Procesos
    proceso_requests = None
    procesos_base_de_datos = []
    
    # Medias
    media_request = None
    medias_base_de_datos = []

    def __init__(self, media_request, medias_base_de_datos):
        self.env = simpy.Environment()
        self.media_requests = media_request
        self.medias_base_de_datos = medias_base_de_datos
        
    def run(self): 
        # Proceso Requets
        self.proceso_requests = Requests(self.env, self.media_requests, self.requests, self.requests_procesadas)
        self.env.process(self.proceso_requests.run())
        
        # Proceso Base de Datos
        for media in self.medias_base_de_datos:
            proceso_base_de_datos = BaseDeDatos(self.env, media, self.requests, self.requests_procesadas)
            self.env.process(proceso_base_de_datos.run())
            self.procesos_base_de_datos.append(proceso_base_de_datos)
        
        self.env.run()
        
    def get_media(self):
        duracion_total = 0
        for request in self.requests_procesadas:
            duracion = request.end_timestamp - request.start_timestamp 
            duracion_total += duracion
        return duracion_total / len(self.requests_procesadas)

### Dos Canales

Canal 1:

$$
\mu_1= 0.7
$$

Canal 2:

$$
\mu_2 = 1
$$

In [58]:
media_requests = 4
media_base_de_datos_1 = 1
media_base_de_datos_2 = 1/0.3
web_service = WebService(media_requests, [media_base_de_datos_1, media_base_de_datos_2])
web_service.run()

In [59]:
web_service.get_media()

0.956059120758693

### Un Canal

Tiempo de servicio es 0.8 segundos, esta es la tasa de servicio.

Unico Canal:

$$
\mu = 0.8
$$

In [30]:
media_requests = 4
media_base_de_datos = 0.8
web_service = WebService(media_requests, [media_base_de_datos])
web_service.run()

### Tiempo medio

#### Simulacion

In [31]:
web_service.get_media()

0.9951420309966112

#### Teorico

In [28]:
def media_teorica(mu, lambda_):
    return 1/(mu - lambda_)

In [29]:
lambda_ = 1/media_requests
mu = 1/media_base_de_datos

media_teorica(mu, lambda_)

1.0

### Fracion que no esperaron

In [ ]:
web_service.count_empty / web_service.n

### Distintos escenarios

In [ ]:
media_requests = 5
media_procesamiento = 10

env = simpy.Environment()
web_service = WebService(env, media_requests, media_procesamiento)
env.process(web_service.generador_requests())
env.process(web_service.generador_procesamiento())
env.run()

print("media simulada: {}".format(web_service.get_media()))

mu = 1/media_procesamiento
lambda_ = 1/media_requests

print("media teorica: {}".format(media_teorica(mu, lambda_)))

## Ejercicio 4

In [ ]:
CAP_MAX = 2000 # capacidad máxima de billetes

MEDIA_ARRIVO = 10 # media de la dist exp de tpo entre arrivo (10min)
LAMBDA_ARRIVO = 1 / MEDIA_ARRIVO

PROB_RET = 0.75 # probabilidad de que el cliente retire
MEDIA_RET = 1.5 # media de la dist exp de tpo de retiro (90s = 1.5min)
LAMBDA_RET = 1 / MEDIA_RET
MIN_RET = 3 # cant min de billetes que retira
MAX_RET = 50 # cant max de billetes que retira

MEDIA_DEP = 5 # media de la dist exp de tpo de deposito (5min)
LAMBDA_DEP = 1 / MEDIA_DEP
MIN_DEP = 10 # cant min de billetes que deposita
MAX_DEP = 110 # cant max de billetes que deposita

DIAS_SIM = 1000
MINUTOS_DIA = 24 * 60
TPO_SIM = DIAS_SIM * MINUTOS_DIA # tiempo de simulación (1000 dias)

In [ ]:
# Definimos la simulación
#
# Referencias y materiales:
# - https://simpy.readthedocs.io/en/latest/simpy_intro/index.html
# - https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.expon.html
# - https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.uniform.html#scipy.stats.uniform
# - https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.randint.html
# - https://towardsdatascience.com/introduction-to-simulation-with-simpy-322606d4ba0c
# - https://simpy.readthedocs.io/en/latest/topical_guides/resources.html#containers

def operar(env, atm, nro_cl, media_op, cantidad, condicion, operacion):
    global caja, tot_abandonos, txs
    with atm.request() as req:
        tpo_entra_queue = env.now
        yield req;
        tpo_sale_queue = env.now
        tpo_espera = tpo_sale_queue - tpo_entra_queue

        tpo_op = expon.rvs(scale=media_op, size=1)[0]

        if condicion(caja, cantidad):
            caja = operacion(caja, cantidad)
            yield env.timeout(tpo_op)

            txs = np.vstack((txs, [caja, env.now, floor(env.now // MINUTOS_DIA)]))
        else:
            # no se da la condición para poder operar, abandona
            tot_abandonos += 1

        tpos_en_sist.append(tpo_espera + tpo_op)

def ret(env, atm, nro_cl):
    cantidad = randint.rvs(MIN_RET, MAX_RET, size=1)[0]
    condicion = lambda caja, cant: caja - cant >= 0
    operacion = lambda caja, cant: caja - cant
    return operar(env, atm, nro_cl, MEDIA_RET, cantidad, condicion, operacion)
    
def dep(env, atm, nro_cl):
    cantidad = randint.rvs(MIN_DEP, MAX_DEP, size=1)[0]
    condicion = lambda caja, cant: caja + cant < CAP_MAX
    operacion = lambda caja, cant: caja + cant
    return operar(env, atm, nro_cl, MEDIA_DEP, cantidad, condicion, operacion)

def arrivos_clientes(env, atm):
    global caja, tot_arrivos
    caja = CAP_MAX

    nro_cl = 0
    while True:
        # generamos un tpo de arrivo exponencial
        tpo_arrivo = expon.rvs(scale=MEDIA_ARRIVO, size=1)[0]
        yield env.timeout(tpo_arrivo)
        nro_cl += 1

        retira_dinero = uniform.rvs() < PROB_RET

        if retira_dinero:
            env.process(ret(env, atm, nro_cl))
        else:
            env.process(dep(env, atm, nro_cl))
    
        tot_arrivos = nro_cl

In [ ]:
# a. simulación

# no uso Container de simpy porque las operaciones put/get no se ajustan al modelo
# y está garantizado que solo se usa un cajero, por lo que el recurso actua de lock
caja = CAP_MAX

# columnas: [monto de caja al finalizar tx; tpo al finalizar tx; dia de la tx]
COL_CAJA = 0
COL_FIN_TX = 1
COL_DIA_TX = 2
txs = np.empty((1, 3))

# tpos_en_sist contempla también el tiempo de aquellos que abandonan al no poder operar
tpos_en_sist = []

tot_abandonos = 0
tot_arrivos = 0

env = simpy.Environment()
atm = simpy.Resource(env, capacity=1)
env.process(arrivos_clientes(env, atm))
env.run(until=TPO_SIM)

In [ ]:
# b. Graficamos para un día particular, indexado desde 0
# https://stackoverflow.com/a/58079385
DIA = 54
txs_dia = txs[txs[:,COL_DIA_TX] == DIA, :]

plt.step(txs_dia[:,COL_FIN_TX] % MINUTOS_DIA, txs_dia[:,COL_CAJA])
plt.xlabel('Tiempo (minutos del dia %d)' % DIA)
plt.ylabel('Caja')
plt.title('Variación de caja en día %d' % DIA)
plt.show()

In [ ]:
# también es de interés ver el comportamiento en los primeros dias
# graficamos para los primeros 5 (0 a 4)
DIA_MAX = 5
txs_dia = txs[txs[:,COL_DIA_TX] < DIA_MAX, :]

plt.step(txs_dia[:,COL_FIN_TX], txs_dia[:,COL_CAJA])
plt.xlabel('Tiempo (minutos desde t=0)')
plt.ylabel('Caja')
plt.title('Variación de caja en los primeros %d dias' % DIA_MAX)
plt.show()

In [ ]:
# c. tpo medio en el sistema
avg = np.average(np.array(tpos_en_sist))
avg_str = str(round(avg, 2))
print('El tiempo promedio en el sistema (espera + uso de cajero) es de %s minutos' % avg_str)

In [ ]:
# d. recomendación a la entidad
UMBRAL_TASA_ABANDONOS = 0.2
tasa_abandonos = tot_abandonos / tot_arrivos
tasa_abandonos_str = str(round(tasa_abandonos * 100, 2))

if tasa_abandonos < UMBRAL_TASA_ABANDONOS:
    print('La simulación resulta en una tasa de abandonos del %s%%. En consecuencia se sugiere la implementación del cambio' % tasa_abandonos_str)
else:
    print('La simulación resulta en una tasa de abandonos del %s%%. En consecuencia no se recomienda la implementación del cambio' % tasa_abandonos_str)

Nota: en casi todas las simulaciones realizadas se obtuvo resultados entre 13% y 15% de tasa de abandono, que es una mejora respecto al 20% inicial.